# Preparation

In [ ]:
import scanpy as sc
import numpy as np
import os
import anndata2ri
import pathlib
import scvelo as scv
from scipy import io
import anndata#
import pandas as pd
from tqdm import tqdm
import argparse
import sys
import re
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")

outdir = "/home/hieunguyen/CRC1382/outdir"
PROJECT = "SAlBounny_full"
path_to_loom_data = os.path.join(outdir, PROJECT, "velocyto_output")
output_version = "20241021"

object_name = "SAlBounny_full.filter_contaminated_cells.clusterRes_0.5_subcluster_gene_Izumo1r"

path_to_main_output = os.path.join(outdir, PROJECT, output_version, "data_analysis")
path_to_01_output = os.path.join(path_to_main_output, "01_output")
path_to_02_output = os.path.join(path_to_main_output, "02_output")
path_to_03_output = os.path.join(path_to_main_output, "03_output")
path_to_08_output = os.path.join(path_to_main_output, "08_output")

path_to_seurat2anndata = os.path.join(path_to_08_output, "seurat2anndata")

#####------------------------------------------------------------#####
##### READ LOOM DATA
#####------------------------------------------------------------#####
all_looms = [item for item in pathlib.Path(path_to_loom_data).glob("*.loom")]

velodata_dict = dict()
for input_loom in tqdm(all_looms):
    velodata = scv.read_loom(input_loom)
    if "200709" in input_loom.name:
        samplename = input_loom.name.replace("SAlBounny_200709_", "").replace(".loom", "")
    else:
        samplename = input_loom.name.replace("SAlBounny_", "").replace(".loom", "")

    new_obs = ["{}_{}_{}-1".format(samplename, samplename, item.split(":")[1].replace("x", "")) for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata.obs["barcode"] = new_obs
    velodata.var_names_make_unique()
    pattern = re.compile('_[A|T|G|C]*-')
    new_obs = [samplename + pattern.search(string = item).group() + item.split("-")[1]
              for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata_dict[samplename] = velodata
    velodata.var_names_make_unique()

all_velodata = velodata_dict[list(velodata_dict.keys())[0]]
for data in list(velodata_dict.keys())[1:]:
    all_velodata = all_velodata.concatenate(velodata_dict[data])
    
new_obs = [item.split("-")[0] + "-1" for item in all_velodata.obs.index]
all_velodata.obs.index = new_obs
all_velodata.var_names_make_unique()

#####------------------------------------------------------------#####
##### READ ANNDATA
#####------------------------------------------------------------#####
adata = sc.read_h5ad(os.path.join(path_to_seurat2anndata, '{}.h5ad'.format(object_name)))

# Main analysis

## Pre-processing pipeline, prepare for RNA velocity analysis

In [ ]:
scv.utils.clean_obs_names(adata)
scv.utils.clean_obs_names(all_velodata)
merge_data = scv.utils.merge(adata, all_velodata)

scv.pp.filter_genes(merge_data, min_shared_counts=20)
scv.pp.normalize_per_cell(merge_data)
scv.pp.filter_genes_dispersion(merge_data, n_top_genes=2000)
scv.pp.log1p(merge_data)

scv.pp.filter_and_normalize(merge_data, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(merge_data, n_pcs=30, n_neighbors=30)

## RNA velocity inference

In [ ]:
# scv.tl.recover_dynamics(merge_data, n_jobs = 15)
# scv.tl.velocity(merge_data, mode="dynamical", diff_kinetics = True)
scv.tl.velocity(merge_data)
scv.tl.velocity_graph(merge_data)

## RNA velocity on UMAP

In [ ]:
colordf = pd.read_csv("/home/hieunguyen/CRC1382/outdir/SAlBounny_full/20241021/data_analysis/06_output/colordf.csv", index_col = [0])
colordf.set_index("V1", inplace = True)
colordf =  colordf.loc[['Anergic', 
                       'Early activation/Trm', 
                       'IFN_1', 
                       'Intermediate', 
                       'Late activation', 
                       'Naive/Tcm', 
                       'Proliferation', 
                       'Tregs']]

colors = colordf["color"].to_list()

In [ ]:
scv.pl.velocity_embedding(merge_data, 
                          dpi=120, 
                          arrow_size=2, 
                          arrow_length=10, 
                          basis = "X_umap",
                          color="celltype",
                         figsize = (12, 12), 
                         fontsize=20, 
                         legend_fontsize=40, 
                         palette = colors,  
                         frameon=True)

## Streamlines

In [ ]:
scv.pl.velocity_embedding_stream(merge_data, 
                                 dpi=120, 
                                 arrow_size=2, basis = "X_umap",
                                 color="celltype",
                         figsize = (12, 12), 
                         fontsize=20, 
                         legend_fontsize = 30, 
                         frameon=True, 
                         palette = colors, 
                         save="streamline_{}.svg".format(object_name))

# Interpretation of the RNA velocities

## Gene ranking



We can test which genes have cluster-specific differential velocity expression, being siginificantly higher/lower compared to the remaining population. Run a differential velocity t-test and outpus a gene ranking for each cluster. Thresholds can be set (e.g. min_corr) to restrict the test on a selection of gene candidates.

In [ ]:
scv.tl.rank_velocity_genes(merge_data, groupby='seurat_clusters', min_corr=.3)

df_gene_ranking = scv.DataFrame(merge_data.uns['rank_velocity_genes']['names'])

### Display top-10 genes for each group (by cell clusters)

In [ ]:
display(df_gene_ranking.head(10))

## Pseudotime

This pseudotime was constructed by a velocity graph

In [ ]:
scv.tl.velocity_pseudotime(merge_data)
scv.pl.scatter(merge_data, color='velocity_pseudotime', cmap='gnuplot', basis = "X_umap",
               figsize = (15, 15), fontsize=20, legend_fontsize = 30, frameon=True)

## Speed and coherence

Two more useful stats: 

- The speed or rate of differentiation is given by the length of the velocity vector. 

- The coherence of the vector field (i.e., how a velocity vector correlates with its neighboring velocities) provides a measure of confidence.

In [ ]:
scv.tl.velocity_confidence(merge_data)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(merge_data, c=keys, cmap='coolwarm', perc=[5, 95], basis = "X_umap", frameon=True, 
              figsize = (10, 10))

In [ ]:
df = merge_data.obs.groupby('seurat_clusters')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

In [ ]:
all_velodata

In [ ]:
adata

In [ ]:
tmpdf = merge_data.var
tmpdf[tmpdf["Chromosome"] == "JH584304.1"]